<a href="https://colab.research.google.com/github/aaalexlit/big-data-hadoop-spark-edx-course/blob/main/course_2_custom_distributed_training/Week4_OneDeviceStrategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### One Device Strategy is typically used to deliberately test your code on a single device

This can be used before switching to a different strategy that distributes across multiple devices

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

### List available devices

In [3]:
devices = tf.config.list_physical_devices(('GPU'))
devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
gpu_name = 'GPU:0'

one_device_strategy = tf.distribute.OneDeviceStrategy(device=gpu_name)

## Params

In [5]:
pixels = 224
RESNET_URL = 'https://tfhub.dev/tensorflow/resnet_50/feature_vector/1'
IMAGE_SIZE = (pixels, pixels)
BATCH_SIZE = 32

print(f"Using {RESNET_URL} with input size {IMAGE_SIZE}")

Using https://tfhub.dev/tensorflow/resnet_50/feature_vector/1 with input size (224, 224)


## Download and Prepare the Dataset
[Cats vs Dogs](https://www.tensorflow.org/datasets/catalog/cats_vs_dogs) dataset

In [7]:
splits = ['train[:80%]', 'train[80%:90%]', 'train[90%:]']

(train_examples, validation_examples, test_examples), info = \
tfds.load('cats_vs_dogs', with_info=True, as_supervised=True, split=splits)

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incompleteJ2Z1RB/cats_vs_dogs-train.tfrecord*...:   0%|…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


In [8]:
# resize the image and normalize pixel values
def format_image(image, label):
    image = tf.image.resize(image, IMAGE_SIZE) / 255.0
    return  image, label

In [11]:
print(num_examples)
print((num_examples // 4)*4)
num_examples // 4


23262
23260


5815

In [12]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [13]:
for image_batch, label_batch in train_batches.take(1):
    pass

print(image_batch.shape)

(32, 224, 224, 3)


## Define and Configure the Model


In [14]:
def build_and_compile_model(do_fine_tuning = False):
    print("Building model with", RESNET_URL)

    # configures the feature extractor fetched from TF Hub
    feature_extractor = hub.KerasLayer(RESNET_URL,
                                   input_shape=IMAGE_SIZE + (3,), 
                                   trainable=do_fine_tuning)

    # define the model
    model = tf.keras.Sequential([
      feature_extractor,
      # append a dense with softmax for the number of classes
      tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

    # display summary
    model.summary()

    # configure the optimizer, loss and metrics
    optimizer = tf.keras.optimizers.SGD(lr=0.002, momentum=0.9) if do_fine_tuning else 'adam'
    model.compile(optimizer=optimizer,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    return model

In [16]:
# build and compile under the strategy scope
with one_device_strategy.scope():
    model = build_and_compile_model()

Building model with https://tfhub.dev/tensorflow/resnet_50/feature_vector/1
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 2048)              23561152  
                                                                 
 dense (Dense)               (None, 2)                 4098      
                                                                 
Total params: 23,565,250
Trainable params: 4,098
Non-trainable params: 23,561,152
_________________________________________________________________


In [17]:
EPOCHS = 5
hist = model.fit(train_batches,
                 epochs=EPOCHS,
                 validation_data=validation_batches)

Epoch 1/5
582/582 [==============================] - 84s 112ms/step - loss: 0.0380 - accuracy: 0.9866 - val_loss: 0.0330 - val_accuracy: 0.9905
Epoch 2/5
582/582 [==============================] - 67s 108ms/step - loss: 0.0210 - accuracy: 0.9936 - val_loss: 0.0324 - val_accuracy: 0.9918
Epoch 3/5
582/582 [==============================] - 67s 108ms/step - loss: 0.0157 - accuracy: 0.9948 - val_loss: 0.0331 - val_accuracy: 0.9905
Epoch 4/5
582/582 [==============================] - 68s 108ms/step - loss: 0.0128 - accuracy: 0.9952 - val_loss: 0.0315 - val_accuracy: 0.9910
Epoch 5/5
582/582 [==============================] - 67s 108ms/step - loss: 0.0099 - accuracy: 0.9968 - val_loss: 0.0435 - val_accuracy: 0.9901
